Rui Hu https://colab.research.google.com/drive/1tSJ9Lj30FnJMwo2JODJDWcf-dSnxOcd4?usp=sharing

**0. Tidy Data Principal**


1. Column names need to be informative, variable names and not values
2. Each column needs to consist of one and only one variable
3. Variables need to be in cells, not rows and columns
4. Each table column needs to have a singular data type
5. A single observational units must be in one table




**1. Import Data from Excel File**


> Separately import different tab

> Easy to control header location







In [38]:
import pandas as pd
import numpy as np
import copy

from google.colab import drive
drive.mount('/content/drive')

# import data from excel, skip top of the table and import multi-index headers
df_content = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'CONTENTS', skiprows = 14, header=[1])
df_1_ori = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 1', skiprows = 13, header=[1,2])
df_2_ori = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 2', skiprows = 13, header=[1,2])
df_3_ori = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 3', skiprows = 13, header=[1,2])
df_4_ori = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 4', skiprows = 13, header=[1,2])
df_5_ori = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 5', skiprows = 13, header=[1,2])
df_6_ori = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name = 'Table 6', skiprows = 13, header=[1,2])

df_country = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name='ANNEX', skiprows = 13, header = [1])
df_notes = pd.read_excel('/content/drive/My Drive/UN_MigrantStockTotal_2015.xlsx',sheet_name='NOTES', skiprows = 13, header = [1])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Sample Input DataFrame**

In [39]:
df_1_ori.head(10)

Sort\norder Major area, region, country or area of destination  \
  Unnamed: 0_level_1                                 Unnamed: 1_level_1   
0                  1                                              WORLD   
1                  2                                  Developed regions   
2                  3                                 Developing regions   
3                  4                          Least developed countries   
4                  5  Less developed regions excluding least develop...   
5                  6                                 Sub-Saharan Africa   
6                  7                                             Africa   
7                  8                                     Eastern Africa   
8                  9                                            Burundi   
9                 10                                            Comoros   

               Notes       Country code   Type of data (a)  \
  Unnamed: 2_level_1 Unnamed: 3_level_1 Unnamed: 4_level_1   
0                NaN                900                NaN   
1                (b)                901                NaN   
2                (c)                902                NaN   
3                (d)                941                NaN   
4                NaN                934                NaN   
5                (e)                947                NaN   
6                NaN                903                NaN   
7                NaN                910                NaN   
8                NaN                108                B R   
9                NaN                174                  B   

  International migrant stock at mid-year (both sexes)                        \
                                                  1990       1995       2000   
0                                          152563212    160801752  172703309   
1                                           82378628     92306854  103375363   
2                                           70184584     68494898   69327946   
3                                           11075966     11711703   10077824   
4                                           59105261     56778501   59244124   
5                                           14690319     15324570   13716539   
6                                           15690623     16352814   14800306   
7                                            5964031      5022742    4844795   
8                                             333110       254853     125628   
9                                              14079        13939      13799   

                         ... International migrant stock at mid-year (male)  \
        2005       2010  ...                                           2000   
0  191269100  221714243  ...                                       87884839   
1  117181109  132560325  ...                                       50536796   
2   74087991   89153918  ...                                       37348043   
3    9809634   10018128  ...                                        5361902   
4   64272611   79130668  ...                                       31986141   
5   13951086   15496764  ...                                        7210452   
6   15191146   16840014  ...                                        7856358   
7    4745792    4657063  ...                                        2480584   
8     172874     235259  ...                                          61094   
9      13209      12618  ...                                           6511   

                                   \
       2005       2010       2015   
0  97866674  114613714  126115435   
1  57217777   64081077   67618619   
2  40648897   50532637   58496816   
3   5383009    5462714    6463217   
4  35265888   45069923   52033599   
5   7444048    8188581   10099486   
6   8231437    9039314   11123423   
7   2529460    2366216    3109176   
8     84805     115823     141311   
9      6286       6060       6071   

  Intern

**2. Function used for Data Cleaning**


> Most of dataframes have similar structures, thus requires similar operations for data cleaning => one generic function **melt_df**
*   Input: DataFrame, DataFrame, String
*   Output: DataFrame, DataFrame

> Table 6 contains three type of data fields, thus needs extra steps => **melt6_df**
*   Input: DataFrame, DataFrame
*   Output: DataFrame, DataFrame

> All tables needs separation of country and major areas data => **split_df**
*   Input: DataFrame, DataFrame
*   Output: DataFrame, DataFrame





In [42]:
#from google.colab import drive
#drive.mount('/content/drive')


# function for general cleaning for table 1,2,3,5
def melt_df(df, df_country, name):
  # check id_vars and value_vars
  id_vars = [idv for idv in df.columns if 'Unnamed' in str(idv[1])]
  value_vars = [valv for valv in df.columns if 'Unnamed' not in str(valv[1])]
  # df.melt, multi-level
  df_multiidx = df.melt(id_vars=id_vars[1:], value_vars=value_vars,var_name=['Gender','Year'], value_name = name)
  # rename df
  df_multiidx = df_multiidx.rename(columns = {col : col[0] for col in id_vars})
  # replace missing value with NaN
  df_multiidx= df_multiidx.replace('..', np.nan)
  # replace gender information
  df_multiidx.loc[df_multiidx.Gender.str.contains("both sexes"),'Gender'] = "Both"
  df_multiidx.loc[df_multiidx.Gender.str.contains("female"),'Gender'] = "F"
  df_multiidx.loc[df_multiidx.Gender.str.contains("male"),'Gender'] = "M"

  return split_df(df_multiidx,df_country)

# specific function for table 6 => three columns of values instead of one
def melt6_df(df,df_country):
  # check id_vars and value_vars
  id_vars = [idv for idv in df.columns if 'Unnamed' in str(idv[1])]
  value_vars1 = [valv for valv in df.columns if ('Unnamed' not in str(valv[1])) and ('-' not in str(valv[1]))]
  value_vars2 = [valv for valv in df.columns if ('Unnamed' not in str(valv[1])) and ('-' in str(valv[1]))]
  # df.melt, multi-level, then pivot back to multi-column values
  df_1 = df.melt(id_vars= id_vars[1:], value_vars=value_vars1,var_name=['Type','Year'], value_name = 'value1').pivot(index = id_vars[1:] + ['Year'], columns = ['Type'], values = 'value1').reset_index()
  df_2 = df.melt(id_vars= id_vars[1:], value_vars=value_vars2,var_name=['Type','Year'], value_name = 'value1').pivot(index = id_vars[1:] + ['Year'], columns = ['Type'], values = 'value1').reset_index()
  # rename df
  df_1,df_2 = df_1.rename(columns = {col : col[0] for col in id_vars}), df_2.rename(columns = {col : col[0] for col in id_vars})
  # replace missing value with NaN
  df_1,df_2= df_1.replace('..', np.nan), df_2.replace('..',np.nan)

  df_a, df_b = split_df(df_1,df_country)
  df_c, df_d = split_df(df_2,df_country)
  return df_a, df_b, df_c, df_d

# split area into country and major area => based on ANNEX table
def split_df(df,df_country):
  df["isCountry"] = [i in list(df_country.iloc[:,1]) for i in df.iloc[:,0]]
  df_c = df[df["isCountry"] == True].rename(columns = {'Major area, region, country or area of destination':'Country'})
  df_a = df[df["isCountry"] == False].rename(columns = {'Major area, region, country or area of destination':'Major Area'})
  df_c, df_a = df_c.drop("isCountry",axis = 1), df_a.drop("isCountry",axis = 1)
  return df_c, df_a

def split_gender(df):
  df_b = df.loc[df["Gender"] == "Both",:].drop('Gender',axis = 1)
  df_g = df.loc[df["Gender"] != "Both",:]
  df_b = df_b.reset_index().drop('index',axis = 1)
  df_g = df_g.reset_index().drop('index',axis = 1)
  return df_b, df_g

**3. Output DataFrame**

In [43]:
df_1_c, df_1_a = melt_df(df_1_ori, df_country, "International migrant stock at mid-year")
df_2_c, df_2_a= melt_df(df_2_ori, df_country, "Total population at mid-year (thousands)")
df_3_c, df_3_a = melt_df(df_3_ori, df_country, "International migrant stock as a percentage of the total population")
df_4_c, df_4_a= melt_df(df_4_ori, df_country, "Female migrants as a percentage of the international migrant stock ")
df_5_c, df_5_a = melt_df(df_5_ori, df_country, "Annual rate of change of the migrant stock")
df_6a_c, df_6a_a, df_6b_c, df_6b_a = melt6_df(df_6_ori, df_country)
# drop gender columns for table 4
df_4_c, df_4_a = df_4_c.drop('Gender',axis = 1), df_4_a.drop('Gender',axis = 1)

**4. Split Dataframe by Gender**

In [44]:
# Table 4 and Table 6 don't have gender separation
df_1_c_both, df_1_c_mf = split_gender(df_1_c)
df_1_a_both, df_1_a_mf = split_gender(df_1_a)
df_2_c_both, df_2_c_mf = split_gender(df_2_c)
df_2_a_both, df_2_a_mf = split_gender(df_2_a)
df_3_c_both, df_3_c_mf = split_gender(df_3_c)
df_3_a_both, df_3_a_mf = split_gender(df_3_a)
df_5_c_both, df_5_c_mf = split_gender(df_5_c)
df_5_a_both, df_5_a_mf = split_gender(df_5_a)

**5. Format Data Field for demonstration**

In [45]:
# use to format value column, "N" for whole number and "P" for percentage
def format_df(df,ty):
  name = df.columns[-1]
  df1 = copy.deepcopy(df)
  if ty == "N": df1[name] = df1[name].map('{:,.0f}'.format)
  elif ty == "P": df1[name] = df1[name].map('{:.2f}%'.format)
  elif ty == "S": 
    df1[name] = df1[name].map('{:.2f}%'.format)
    df1[df1.columns[-2]] = df1[df1.columns[-2]].map('{:,.0f}'.format)
  return df1

**Table of Contents**

In [46]:
df_content

,TABLE,TITLE
0,Table 1,International migrant stock at mid-year by sex...
1,Table 2,Total population at mid-year by sex and by maj...
2,Table 3,International migrant stock as a percentage of...
3,Table 4,Female migrants as a percentage of the interna...
4,Table 5,Annual rate of change of the migrant stock by ...
5,Table 6,Estimated refugee stock at mid-year by major a...
6,ANNEX,Classification of countries and areas by major...
7,NOTES,NOTES


**Table 1: 	International migrant stock at mid-year by sex and by major area, region, country or area, 1990-2015**

In [47]:
format_df(df_1_c_both,"N").head(5)

,Country,Notes,Country code,Type of data (a),Year,International migrant stock at mid-year
0,Burundi,NaN,108,B R,1990,"333,110"
1,Comoros,NaN,174,B,1990,"14,079"
2,Djibouti,NaN,262,B R,1990,"122,221"
3,Eritrea,NaN,232,I,1990,"11,848"
4,Ethiopia,NaN,231,B R,1990,"1,155,390"


In [48]:
format_df(df_1_a_both,"N").head(5)

,Major Area,Notes,Country code,Type of data (a),Year,International migrant stock at mid-year
0,WORLD,NaN,900,NaN,1990,"152,563,212"
1,Developed regions,(b),901,NaN,1990,"82,378,628"
2,Developing regions,(c),902,NaN,1990,"70,184,584"
3,Least developed countries,(d),941,NaN,1990,"11,075,966"
4,Less developed regions excluding least develop...,NaN,934,NaN,1990,"59,105,261"


In [49]:
format_df(df_1_c_mf,"N").head(5)

,Country,Notes,Country code,Type of data (a),Gender,Year,International migrant stock at mid-year
0,Burundi,NaN,108,B R,M,1990,"163,267"
1,Comoros,NaN,174,B,M,1990,"6,717"
2,Djibouti,NaN,262,B R,M,1990,"64,242"
3,Eritrea,NaN,232,I,M,1990,"6,228"
4,Ethiopia,NaN,231,B R,M,1990,"607,284"


In [50]:
format_df(df_1_a_mf,"N").head(5)

,Major Area,Notes,Country code,Type of data (a),Gender,Year,International migrant stock at mid-year
0,WORLD,NaN,900,NaN,M,1990,"77,747,510"
1,Developed regions,(b),901,NaN,M,1990,"40,263,397"
2,Developing regions,(c),902,NaN,M,1990,"37,484,113"
3,Least developed countries,(d),941,NaN,M,1990,"5,843,107"
4,Less developed regions excluding least develop...,NaN,934,NaN,M,1990,"31,641,006"


**Table 2: Total population at mid-year by sex and by major area, region, country or area, 1990-2015 (thousands)**

In [51]:
format_df(df_2_c_both,"N").head(5)

,Country,Notes,Country code,Year,Total population at mid-year (thousands)
0,Burundi,NaN,108,1990,"5,613"
1,Comoros,NaN,174,1990,415
2,Djibouti,NaN,262,1990,588
3,Eritrea,NaN,232,1990,"3,139"
4,Ethiopia,NaN,231,1990,"48,057"


In [52]:

format_df(df_2_a_both,"N").head(5)

,Major Area,Notes,Country code,Year,Total population at mid-year (thousands)
0,WORLD,NaN,900,1990,"5,309,668"
1,Developed regions,(b),901,1990,"1,144,463"
2,Developing regions,(c),902,1990,"4,165,205"
3,Least developed countries,(d),941,1990,"510,058"
4,Less developed regions excluding least develop...,NaN,934,1990,"3,655,147"


In [53]:
format_df(df_2_c_mf,"N").head(5)

,Country,Notes,Country code,Gender,Year,Total population at mid-year (thousands)
0,Burundi,NaN,108,M,1990,"2,755"
1,Comoros,NaN,174,M,1990,208
2,Djibouti,NaN,262,M,1990,296
3,Eritrea,NaN,232,M,1990,"1,558"
4,Ethiopia,NaN,231,M,1990,"23,966"


In [54]:
format_df(df_2_a_mf,"N").head(5)

,Major Area,Notes,Country code,Gender,Year,Total population at mid-year (thousands)
0,WORLD,NaN,900,M,1990,"2,670,424"
1,Developed regions,(b),901,M,1990,"555,256"
2,Developing regions,(c),902,M,1990,"2,115,168"
3,Least developed countries,(d),941,M,1990,"254,043"
4,Less developed regions excluding least develop...,NaN,934,M,1990,"1,861,126"


**Table 3:	International migrant stock as a percentage of the total population, 1990-2015**

In [55]:
format_df(df_3_c_both,"P").head(5)

,Country,Notes,Country code,Type of data (a),Year,International migrant stock as a percentage of the total population
0,Burundi,NaN,108,B R,1990,5.93%
1,Comoros,NaN,174,B,1990,3.39%
2,Djibouti,NaN,262,B R,1990,20.77%
3,Eritrea,NaN,232,I,1990,0.38%
4,Ethiopia,NaN,231,B R,1990,2.40%


In [56]:
format_df(df_3_a_both,"P").head(5)

,Major Area,Notes,Country code,Type of data (a),Year,International migrant stock as a percentage of the total population
0,WORLD,NaN,900,NaN,1990,2.87%
1,Developed regions,(b),901,NaN,1990,7.20%
2,Developing regions,(c),902,NaN,1990,1.69%
3,Least developed countries,(d),941,NaN,1990,2.17%
4,Less developed regions excluding least develop...,NaN,934,NaN,1990,1.62%


In [57]:
format_df(df_3_c_mf,"P").head(5)

,Country,Notes,Country code,Type of data (a),Gender,Year,International migrant stock as a percentage of the total population
0,Burundi,NaN,108,B R,M,1990,5.93%
1,Comoros,NaN,174,B,M,1990,3.23%
2,Djibouti,NaN,262,B R,M,1990,21.71%
3,Eritrea,NaN,232,I,M,1990,0.40%
4,Ethiopia,NaN,231,B R,M,1990,2.53%


In [58]:
format_df(df_3_a_mf,"P").head(5)

,Major Area,Notes,Country code,Type of data (a),Gender,Year,International migrant stock as a percentage of the total population
0,WORLD,NaN,900,NaN,M,1990,2.91%
1,Developed regions,(b),901,NaN,M,1990,7.25%
2,Developing regions,(c),902,NaN,M,1990,1.77%
3,Least developed countries,(d),941,NaN,M,1990,2.30%
4,Less developed regions excluding least develop...,NaN,934,NaN,M,1990,1.70%


**Table 4: 	Female migrants as a percentage of the international migrant stock by major area, region, country or area, 1990-2015**

In [59]:
format_df(df_4_c,"P").head(5)

,Country,Notes,Country code,Type of data (a),Year,Female migrants as a percentage of the international migrant stock
8,Burundi,NaN,108,B R,1990,50.99%
9,Comoros,NaN,174,B,1990,52.29%
10,Djibouti,NaN,262,B R,1990,47.44%
11,Eritrea,NaN,232,I,1990,47.43%
12,Ethiopia,NaN,231,B R,1990,47.44%


In [60]:
format_df(df_4_a,"P").head(5)

,Major Area,Notes,Country code,Type of data (a),Year,Female migrants as a percentage of the international migrant stock
0,WORLD,NaN,900,NaN,1990,49.04%
1,Developed regions,(b),901,NaN,1990,51.12%
2,Developing regions,(c),902,NaN,1990,46.59%
3,Least developed countries,(d),941,NaN,1990,47.26%
4,Less developed regions excluding least develop...,NaN,934,NaN,1990,46.47%


**Table 5: 	Annual rate of change of the migrant stock by sex and by major area, region, country or area, 1990-2015 (percentage)**

In [61]:
format_df(df_5_c_both,"P").head(5)

,Country,Notes,Country code,Type of data (a),Year,Annual rate of change of the migrant stock
0,Burundi,NaN,108,B R,1990-1995,-5.36%
1,Comoros,NaN,174,B,1990-1995,-0.20%
2,Djibouti,NaN,262,B R,1990-1995,-4.06%
3,Eritrea,NaN,232,I,1990-1995,0.91%
4,Ethiopia,NaN,231,B R,1990-1995,-7.18%


In [62]:
format_df(df_5_a_both,"P").head(5)

,Major Area,Notes,Country code,Type of data (a),Year,Annual rate of change of the migrant stock
0,WORLD,NaN,900,NaN,1990-1995,1.05%
1,Developed regions,(b),901,NaN,1990-1995,2.28%
2,Developing regions,(c),902,NaN,1990-1995,-0.49%
3,Least developed countries,(d),941,NaN,1990-1995,1.12%
4,Less developed regions excluding least develop...,NaN,934,NaN,1990-1995,-0.80%


In [63]:
format_df(df_5_c_mf,"P").head(5)

,Country,Notes,Country code,Type of data (a),Gender,Year,Annual rate of change of the migrant stock
0,Burundi,NaN,108,B R,M,1990-1995,-5.48%
1,Comoros,NaN,174,B,M,1990-1995,-0.31%
2,Djibouti,NaN,262,B R,M,1990-1995,-4.05%
3,Eritrea,NaN,232,I,M,1990-1995,0.98%
4,Ethiopia,NaN,231,B R,M,1990-1995,-7.18%


In [64]:
format_df(df_5_a_mf,"P").head(5)

,Major Area,Notes,Country code,Type of data (a),Gender,Year,Annual rate of change of the migrant stock
0,WORLD,NaN,900,NaN,M,1990-1995,1.00%
1,Developed regions,(b),901,NaN,M,1990-1995,2.27%
2,Developing regions,(c),902,NaN,M,1990-1995,-0.45%
3,Least developed countries,(d),941,NaN,M,1990-1995,1.00%
4,Less developed regions excluding least develop...,NaN,934,NaN,M,1990-1995,-0.73%


**Table 6a: Estimated refugee stock at mid-year by major area, region, country or area, 1990-2015**

In [65]:
format_df(df_6a_c,"S").head(5)

Type,Country,Notes,Country code,Type of data (a),Year,Estimated refugee stock at mid-year (both sexes),Refugees as a percentage of the international migrant stock
0,Afghanistan,NaN,4,B,1990,25,0.04%
1,Afghanistan,NaN,4,B,1995,"19,605",27.41%
2,Afghanistan,NaN,4,B,2000,0,0.00%
3,Afghanistan,NaN,4,B,2005,32,0.04%
4,Afghanistan,NaN,4,B,2010,"6,434",6.29%


In [66]:
format_df(df_6a_a,"S").head(5)

Type,Major Area,Notes,Country code,Type of data (a),Year,Estimated refugee stock at mid-year (both sexes),Refugees as a percentage of the international migrant stock
6,Africa,NaN,903,NaN,1990,"5,687,352",36.25%
7,Africa,NaN,903,NaN,1995,"5,949,953",36.38%
8,Africa,NaN,903,NaN,2000,"3,609,138",24.39%
9,Africa,NaN,903,NaN,2005,"2,750,644",18.11%
10,Africa,NaN,903,NaN,2010,"2,413,917",14.33%


**Table 6b: Annual rate of change of the refugee stock**

In [67]:
format_df(df_6b_c,"P").head(5)

Type,Country,Notes,Country code,Type of data (a),Year,Annual rate of change of the refugee stock
0,Afghanistan,NaN,4,B,1990-1995,128.99%
1,Afghanistan,NaN,4,B,1995-2000,nan%
2,Afghanistan,NaN,4,B,2000-2005,nan%
3,Afghanistan,NaN,4,B,2005-2010,102.91%
4,Afghanistan,NaN,4,B,2010-2015,50.50%


In [68]:
format_df(df_6b_a,"P").head(5)

Type,Major Area,Notes,Country code,Type of data (a),Year,Annual rate of change of the refugee stock
5,Africa,NaN,903,NaN,1990-1995,0.08%
6,Africa,NaN,903,NaN,1995-2000,-8.00%
7,Africa,NaN,903,NaN,2000-2005,-5.95%
8,Africa,NaN,903,NaN,2005-2010,-4.67%
9,Africa,NaN,903,NaN,2010-2015,6.13%


**Appendix: 	Classification of countries and areas by major area and region**

In [69]:
df_country.head(10)

,Country code,Country or area,Sort order,Major area,Code,Sort order.1,Region,Code.1,Sort order.2,Developed region,Least developed country,Sub-Saharan Africa
0,4,Afghanistan,99,Asia,935,71,Southern Asia,5501,98,No,Yes,No
1,8,Albania,154,Europe,908,127,Southern Europe,925,153,Yes,No,No
2,12,Algeria,40,Africa,903,7,Northern Africa,912,39,No,No,No
3,16,American Samoa,257,Oceania,909,238,Polynesia,957,256,No,No,No
4,20,Andorra,155,Europe,908,127,Southern Europe,925,153,Yes,No,No
5,24,Angola,30,Africa,903,7,Middle Africa,911,29,No,Yes,Yes
6,660,Anguilla,182,Latin America and the Caribbean,904,180,Caribbean,915,181,No,No,No
7,28,Antigua and Barbuda,183,Latin America and the Caribbean,904,180,Caribbean,915,181,No,No,No
8,32,Argentina,218,Latin America and the Caribbean,904,180,South America,931,217,No,No,No
9,51,Armenia,109,Asia,935,71,Western Asia,922,108,No,No,No


**Appendix: NOTES**

In [70]:
df_notes.columns = ["FootNotes", "NOTES"]
df_notes.head(10)

,FootNotes,NOTES
0,(a),The column labeled “Type of data” indicates wh...
1,(b),"More developed regions comprise Europe, Northe..."
2,(c),Less developed regions comprise all regions of...
3,(d),"The least developed countries, as defined by t..."
4,(e),Sub-Saharan Africa refers to all of Africa exc...
5,(1),"Including Agalega, Rodrigues and Saint Brandon."
6,(2),Including Zanzibar.
7,(3),The estimates for 1990 to 2005 refer to Sudan ...
8,(4),Including Ascension and Tristan da Cunha.
9,(5),"For statistical purposes, the data for China d..."
